In [14]:
import requests
from bs4 import BeautifulSoup
import json
import time

from tqdm import tqdm
import re

from urllib.parse import urljoin

In [3]:
# 

In [4]:
def extract_zangdiyg_page_article_links(url, base_url):
    """
    Extracts all article links from a given zangdiyg webpage.

    This function scrapes the provided URL and extracts links to individual articles
    found on the page.

    Args:
    url (str): The URL of the zangdiyg webpage containing article links.

    Returns:
        {
            "Links": List[],
            "Message": string,
            "Response": int,
            "source_url": string
        }
    Raises:
    requests.RequestException: If there's an error fetching the webpage.
    ValueError: If the expected HTML structure is not found on the page.
    """

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    final_response = {
        "Links": [],
        "Message": "Success",
        "Response": 200,
        "source_url": url
    }
    
    try:
        start_time = time.time()
        response = requests.get(url, headers=headers, timeout=(5, 60-5))
        response.raise_for_status()
        end_time = time.time()

        if end_time-start_time > 50:
            print(f"This ULR Took more then 50s: {url}")
        
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # # Getting all the links of articles 
        all_links = []
        all_link_article = soup.find("div", class_="panel-body")
        if all_link_article:
            article_block = all_link_article.find_all("div", class_="media")
            if article_block:
                for each_head in article_block:
                    article_links = each_head.find_all("a")
                    if article_links:
                        for each_link in article_links:
                            full_url = urljoin(base_url, each_link.get("href"))
                            all_links.append(full_url)
                        
        final_response["Links"] = all_links
        return final_response
     
    except requests.Timeout:
        final_response["Message"] = "Request timed out"
        final_response["Response"] = 408  # Request Timeout
        return final_response
    except requests.RequestException as e:
        # print(f"An error occurred while fetching the webpage: {e}")
        final_response["Message"] = f"An error occurred while fetching the webpage: {e}"
        final_response["Response"] = getattr(e.response, 'status_code', None)
        return final_response
    except ValueError as e:
        # print(f"An error occurred while parsing the webpage: {e}")
        final_response["Message"] = f"An error occurred while parsing the webpage: {e}"
        final_response["Response"] = 404
        # getattr(e.response, 'status_code', None)
        return final_response
    except Exception as e:
        # print(f"An unexpected error occurred: {e}")
        final_response["Message"] = f"An unexpected error occurred: {e}"
        final_response["Response"] = 500
        return final_response


In [6]:
base_url = "https://ti.zangdiyg.com"
url = "https://ti.zangdiyg.com/category/index/id/134/p/2.html"
extract_zangdiyg_page_article_links(url, base_url)

{'Links': ['https://ti.zangdiyg.com/article/detail/id/25510.html',
  'https://ti.zangdiyg.com/article/detail/id/25489.html',
  'https://ti.zangdiyg.com/article/detail/id/25448.html',
  'https://ti.zangdiyg.com/article/detail/id/25419.html',
  'https://ti.zangdiyg.com/article/detail/id/25408.html',
  'https://ti.zangdiyg.com/article/detail/id/25381.html',
  'https://ti.zangdiyg.com/article/detail/id/25374.html',
  'https://ti.zangdiyg.com/article/detail/id/25345.html',
  'https://ti.zangdiyg.com/article/detail/id/25325.html',
  'https://ti.zangdiyg.com/article/detail/id/25315.html',
  'https://ti.zangdiyg.com/article/detail/id/25299.html',
  'https://ti.zangdiyg.com/article/detail/id/25269.html',
  'https://ti.zangdiyg.com/article/detail/id/25256.html',
  'https://ti.zangdiyg.com/article/detail/id/25245.html',
  'https://ti.zangdiyg.com/article/detail/id/25239.html',
  'https://ti.zangdiyg.com/article/detail/id/25205.html',
  'https://ti.zangdiyg.com/article/detail/id/25195.html',
  'ht

In [4]:
custom_url = 'https://ti.zangdiyg.com/category/index/id/61.html'
print(custom_url.replace(".html", ""))

https://ti.zangdiyg.com/category/index/id/61


In [23]:

def scrape_zangdiyg_article_content(url, tags):
    """
    
    
    """


    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    
    final_response = {
        "data": {
            'title': "",
            'body': {"Audio": "", "Text": []},
            'meta_data': {'URL': url, 'Author': "", 'Date': "", 'Tags': [tags]}
        },
        "Message": "Success",
        "Response": 200
    }
    
    try:
        # Make the request to the URL
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        
        # Parse the page content with BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')
        tags_body = soup.find("div", class_="panel-title")
        tags = []
        if tags_body:
            tag_list = tags_body.find_all("a")
            if tag_list:
                for tag in tag_list:
                    each_tag = tag.get_text(strip=True)
                    tags.append(each_tag)
                final_response['data']['meta_data']["Tags"] = tags
        
        full_body = soup.find('div', class_="panel-body")
        # print(full_body)
        if full_body:
            # Extract title
            # Find the specific h1 element
            title = soup.find('h1', class_='ti', style=lambda value: value and 'text-align: center' in value and 'font-size:2em' in value)

            if title:
                title_text = title.get_text(strip=True)
            else:
                title_text = ""
            final_response['data']["title"] = title_text
            
            metadata = full_body.find('h4', class_="dit")
            # Extracting Meta Data
            try:
                if metadata:
                    all_text = metadata.get_text(strip=True)
                    if all_text:
                        all_text = re.sub(' +', ' ', all_text)
                        split_text = all_text.split(" ")
                        # Extract date
                        date = split_text[-1]
                        # Extract author
                        author = ' '.join(split_text[:-2])
                        final_response['data']['meta_data']["Date"] = date
                        final_response['data']['meta_data']["Author"] = author
            except AttributeError:
                final_response['data']['meta_data']["Author"] = "Error fetching author"
                final_response['data']['meta_data']["Date"] = "Error fetching date"
            
        # Extract body content
        try:
            body = full_body.find("div", id="contentp")
            if body:
                paragraphs = body.find_all("p")
                if paragraphs:
                    # Extracting all <p> tags for text content
                    final_response['data']['body']["Text"] = [para.get_text(strip=True) for para in paragraphs]
                else:
                    final_response['data']['body']["Text"] = [""]

        except AttributeError as e:
            final_response['data']['body']["Text"] = [f"Error fetching body content{str(e)}"]
        
        return final_response
    except requests.Timeout:
        final_response["Message"] = "Request timed out"
        final_response["Response"] = 408  # Request Timeout
        return final_response
        
    except requests.RequestException as e:
        final_response["Message"] = f"An error occurred while fetching the article: {str(e)}"
        final_response["Response"] = getattr(e.response, 'status_code', 500)
        return final_response
    except Exception as e:
        # print(f"An unexpected error occurred: {e}")
        final_response["Message"] = f"An unexpected error occurred: {e}"
        final_response["Response"] = 500
        return final_response




In [24]:
url = "https://ti.zangdiyg.com/article/detail/id/25510.html"
url = "https://ti.zangdiyg.com/article/detail/id/27133.html"
scrape_zangdiyg_article_content(url, tags="དཔྱད་གཏམ།")

{'data': {'title': 'རྒྱལ་ཡོངས་བོད་སྐད་བརྡ་ཆད་ཚད་ལྡན་ཅན་གྱི་ལས་དོན་ཨུ་ལྷན་གྱི་ཨུ་ལྷན་གཉིས་ཀྱི་སྐབས་བརྗེའི་ཚོགས་འདུ་དང་2023ལོའི་བོད་སྐད་བརྡ་ཆད་གསར་པ་ཞུ་དག་གཏན་འབེབས་ཀྱི་ཚོགས་འདུ་ལྷ་ས་རུ་འཚོགས།',
  'body': {'Audio': '',
   'Text': ['',
    'རྒྱལ་ཡོངས་བོད་སྐད་བརྡ་ཆད་གསར་པ་ཚད་ལྡན་ཅན་དང་ཚད་གཞི་ཅན་གྱི་ཆུ་ཚད་མཐོ་རུ་གཏོང་བ་དང་།\xa0རྒྱལ་ཡོངས་བོད་སྐད་བརྡ་ཆད་ཚད་ལྡན་ཅན་གྱི་ལས་དོན་ཨུ་ཡོན་ལྷན་ཁང་གི་དྲུང་ཡིག་ཁྲུའུ་ཡི་བྱ་བའི་འགན་ནུས་སྔར་བས་འདོན་སྤེལ་བྱ་ཆེད།\xa0སྤྱི་ཟླ་9པའི་ཚེས་7ཉིན་ནས་8ཉིན་བར་རྒྱལ་ཡོངས་བོད་སྐད་བརྡ་ཆད་ཚད་ལྡན་ཅན་གྱི་ལས་དོན་ཨུ་ཡོན་ལྷན་ཁང་གིས་ཨུ་ལྷན་གཉིས་ཀྱི་སྐབས་བརྗེའི་ཚོགས་འདུ་དང་2023ལོའི་བོད་སྐད་བརྡ་ཆད་གསར་པ་ཞུ་དག་གཏན་འབེབས་ཀྱི་ཚོགས་འདུ་ལྷ་ས་རུ་འཚོགས།\xa0ཐེངས་འདིའི་ཚོགས་འདུ་ནི་རྒྱལ་ཡོངས་བོད་སྐད་བརྡ་ཆད་ཚད་ལྡན་ཅན་གྱི་ལས་དོན་ཨུ་ཡོན་ལྷན་ཁང་གིས་གཙོ་སྒྲུབ་བྱས་པ་དང་།\xa0ཀྲུང་གོའི་བོད་རིག་པའི་ཞིབ་འཇུག་ལྟེ་གནས་ཀྱི་ཚན་ཞིབ་ལས་དོན་གཞུང་ལས་ཁང་དང་བོད་རང་སྐྱོང་ལྗོངས་བོད་སྐད་ཡིག་ལས་དོན་ཨུ་ཡོན་ལྷན་ཁང་གི་གཞུང་ལས་ཁང་(རྩོམ་སྒྱུར་ཅུའུ)ཡིས་འགན་སྒྲུབ་བྱས་ཡོད།\xa0བོད་ལྗོངས་དང་བོད་འབྲེལ་བྱ་བའི་གཙོ་གནད་ཞིང་ཆེན་ག